In [1]:
import numpy as np

In [2]:
opcodeInput=np.loadtxt("input_day11.txt",delimiter=",",dtype=np.int64)

In [3]:
class OpCodeManager:
    def __init__(self, opcode):
        self.opcode = opcode
        self.relativeBaseIdx=0
    def __getitem__(self, idx):
        if idx >= self.opcode.size:
            self.opcode.resize(idx+1, refcheck=False) # fill rest with zeros
        return self.opcode[idx]
    def __setitem__(self, idx, value):
        if idx >= self.opcode.size:
            self.opcode.resize(idx+1, refcheck=False) # fill rest with zeros
        self.opcode[idx] = value

In [4]:
def decodeCommand(command):
    code=command%100
    p1Mode=int(command/100)%10
    p2Mode=int(command/1000)%10
    p3Mode=int(command/10000)%10
    return [code, p1Mode, p2Mode, p3Mode]

def getArgument(opcode, mode, parameter):
    if mode == 0:
        # position mode
        return opcode[parameter]
    elif mode == 1:
        # immediate mode
        return parameter
    else:
        # relative mode
        return opcode[opcode.relativeBaseIdx+parameter]
    
def getWritePosition(opcode, mode, parameter):
    if mode == 0:
        # position mode
        return parameter
    elif mode == 1:
        # immediate mode
        print("ERROR! Immediate mode not intended for writing")
        print(mode, parameter, opcode)
    else:
        # relative mode
        return opcode.relativeBaseIdx+parameter

In [11]:
class PaintingRoboter:
    def __init__(self, opcodeMgr):
        self.opcode = opcodeMgr
        self.outputList=[]
        self.paintedPanels={}
        self.position=(0,0)
        self.orientation=0 # 0:top, 1: right, 2:bottom, 3:left
        
    def clearOutput(self):
        self.outputList=[]
    
    def updateOrientation(self,turning):
        orientUpdate = turning
        if turning == 0:
            orientUpdate = -1
        self.orientation = (self.orientation + orientUpdate)%4
        if self.orientation < 0:
            self.orientation += 4
            
    def updatePosition(self):
        if self.orientation == 0:
            self.position = (self.position[0],self.position[1]+1)
        if self.orientation == 1:
            self.position = (self.position[0]+1,self.position[1])
        if self.orientation == 2:
            self.position = (self.position[0],self.position[1]-1)
        if self.orientation == 3:
            self.position = (self.position[0]-1,self.position[1])
    
    def nextStep(self):
        if len(self.outputList) == 0:
            # start up
            # task 1
            #return 0
            # task 2
            return 1
        # read output
        paintColor=self.outputList[0]
        turning=self.outputList[1]
        self.clearOutput()
        
        #update painted panels
        self.paintedPanels[self.position]=paintColor
        
        #move robot
        self.updateOrientation(turning)
        self.updatePosition()
        
        #create new input
        newInput = 0
        if self.position in self.paintedPanels:
            newInput = self.paintedPanels[self.position]
        return newInput

    def run(self, idx):
    
        while True:

            decoded = decodeCommand(self.opcode[idx])

            if decoded[0] == 99:
                print("Finished!")
                return
            elif decoded[0] == 1:
                #add p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                self.opcode[arg3] = arg1 + arg2
                idx+=4
            elif decoded[0] == 2:
                #multiply p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                self.opcode[arg3] = arg1 * arg2
                idx+=4
            elif decoded[0] == 3:
                #input p1
                p1 = self.opcode[idx+1]
                arg1 = getWritePosition(self.opcode,decoded[1],p1)
                # call update function that generates next input
                var = self.nextStep()
                self.opcode[arg1] = np.int64(var)
                idx+=2
            elif decoded[0] == 4:
                #output p1
                p1 = self.opcode[idx+1]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                print("Output: ", arg1)
                self.outputList.append(arg1)
                idx+=2
            elif decoded[0] == 5:
                #jump-if-true p1 p2
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                if arg1 != 0:
                    idx=arg2
                else:      
                    idx=idx+3
            elif decoded[0] == 6:
                #jump-if-false p1 p2
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                if arg1 == 0:
                    idx=arg2
                else:      
                    idx+=3
            elif decoded[0] == 7:
                #less-than p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                if arg1 < arg2:
                    self.opcode[arg3]=1
                else:      
                    self.opcode[arg3]=0
                idx+=4
            elif decoded[0] == 8:
                #equals p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                if arg1 == arg2:
                    self.opcode[arg3]=1
                else:      
                    self.opcode[arg3]=0
                idx+=4
            elif decoded[0] == 9:
                # change relative base p1
                p1 = self.opcode[idx+1]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                self.opcode.relativeBaseIdx += arg1
                idx+=2
            else:
                print("Invalid code ", self.opcode, idx, self.opcode[idx], decoded)
                return

In [9]:
#task 1
opcodeMgr=OpCodeManager(opcodeInput.copy())
robot = PaintingRoboter(opcodeMgr)
robot.run(0)

Output:  1
Output:  0
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1

Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  0
Output:  0
Output:  1
Output:  0
Output:  1
Output:  0
Output:  1
Output:  0
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  0
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  0
Output:  1
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0

Output:  1
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1

Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  0
Output:  1
Output:  0
Output:  1
Output:  1
Output:  0
Output:  0
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  0
Output:  1
Output:  0
Output:  1
Output:  0
Output:  1
Output:  0
Output:  0
Output:  1
Output:  0
Output:  0
Output:  1
Output:  0
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  0
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1

Output:  1
Output:  0
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  0
Output:  1
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  0
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  0
Output:  1
Output:  0
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  0
Output:  1
Output:  1
Output:  1

...

Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  0
Output:  0
Output:  0
Output:  1
Output:  0
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  0
Output:  1
Output:  1
Output:  0
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  0
Output:  1
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  0
Output:  1
Output:  0
Output:  1
Output:  0
Output:  0
Output:  0
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  0
Output:  1
Output:  0
Output:  1

Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  0
Output:  1
Output:  0
Output:  0
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  0
Output:  0
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  0
Output:  1
Output:  0
Output:  1
Output:  0
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  0
Output:  1
Output:  1
Output:  0
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  0

Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  1
Output:  0
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  0
Output:  0
Output:  0
Output:  1
Output:  0
Output:  1
Output:  0
Output:  0
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  0
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1

Output:  1
Output:  0
Output:  0
Output:  1
Output:  0
Output:  1
Output:  0
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  0
Output:  1
Output:  1
Output:  0
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1

Output:  0
Output:  0


In [10]:
print(len(robot.paintedPanels))

1934


In [12]:
#task 2
opcodeMgr=OpCodeManager(opcodeInput.copy())
robot = PaintingRoboter(opcodeMgr)
robot.run(0)

Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  1
Output:  0
Output:  0
Output:  1
Output:  0
Output:  1
Output:  0
Output:  0
Output:  1
Output:  0
Output:  1
Output:  1
Output:  0
Output:  1
Output:  0
Output:  0
Output:  1
Output:  0
Output:  0
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  0
Output:  1
Output:  0
Output:  0
Output:  1
Output:  0
Output:  1
Output:  1
Output:  0
Output:  1
Output:  0
Output:  0
Output:  1
Output:  0
Output:  1
Output:  1
Output:  1
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  1
Output:  0
Output:  1
Output:  1
Output:  0
Output:  0
Output:  0
Output:  0
Output:  1
Output:  0
Output:  1
Output:  1
Output:  0
Output:  0
Output:  0
Output:  1
Output:  1
Output:  1

In [14]:
# convert dict to 2D array
xMin=0
xMax=0
yMin=0
yMax=0
for elem in robot.paintedPanels.keys():
    xMin = min(xMin,elem[0])
    xMax = max(xMax,elem[0])
    yMin = min(yMin,elem[1])
    yMax = max(yMax,elem[1])
print(xMin,xMax,yMin,yMax)

0 42 -5 0


In [26]:
outputArray=np.full((xMax-xMin+1,yMax-yMin+1),' ',str)
outputArray

array([[' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' 

In [27]:
for elem in robot.paintedPanels.items():
    pos=elem[0]
    paint=elem[1]
    if paint == 1:
        outputArray[pos[0]-xMin][pos[1]-yMin]='#'

array([[' ', ' ', ' ', ' ', ' ', ' '],
       ['#', '#', '#', '#', '#', '#'],
       [' ', ' ', '#', ' ', ' ', '#'],
       [' ', '#', '#', ' ', ' ', '#'],
       ['#', ' ', ' ', '#', '#', ' '],
       [' ', ' ', ' ', ' ', ' ', ' '],
       ['#', '#', '#', '#', '#', '#'],
       [' ', ' ', ' ', '#', ' ', ' '],
       [' ', '#', '#', ' ', '#', ' '],
       ['#', ' ', ' ', ' ', ' ', '#'],
       [' ', ' ', ' ', ' ', ' ', ' '],
       [' ', '#', '#', '#', '#', '#'],
       ['#', ' ', ' ', ' ', ' ', ' '],
       ['#', ' ', ' ', ' ', ' ', ' '],
       [' ', '#', '#', '#', '#', '#'],
       [' ', ' ', ' ', ' ', ' ', ' '],
       ['#', '#', '#', '#', '#', '#'],
       [' ', ' ', '#', ' ', ' ', '#'],
       [' ', '#', '#', ' ', ' ', '#'],
       ['#', ' ', ' ', '#', '#', ' '],
       [' ', ' ', ' ', ' ', ' ', ' '],
       [' ', '#', '#', '#', '#', ' '],
       ['#', ' ', ' ', ' ', ' ', '#'],
       ['#', ' ', '#', ' ', ' ', '#'],
       ['#', '#', '#', ' ', '#', ' '],
       [' ', ' ', ' ', ' 

In [31]:
reshapedArray = np.flip(np.transpose(outputArray),0)
outputStr=''
for array in reshapedArray:
    for elem in array:
        outputStr+=elem
    outputStr+='\n'
print(outputStr)

 ###  #  # #  # ###   ##  #  #  ##  #  #   
 #  # # #  #  # #  # #  # # #  #  # # #    
 #  # ##   #  # #  # #    ##   #    ##     
 ###  # #  #  # ###  # ## # #  # ## # #    
 # #  # #  #  # # #  #  # # #  #  # # #    
 #  # #  #  ##  #  #  ### #  #  ### #  #   

